# **라이브러리 로드**

In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# 데이터 전처리 패키지
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 모델 패키지
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding, LSTM
from tensorflow.keras.models import Sequential

# 모델 평가 패키지
from sklearn.metrics import mean_squared_error


# **1. 순환 신경망(Recurrent Neural Network, RNN)**

- 루프(loop)를 가진 신경망의 한 종류

- 시퀀스의 원소를 순회하면서 지금까지 처리한 정보를 상태(state)에 저장

<img src="https://miro.medium.com/max/627/1*go8PHsPNbbV6qRiwpUQ5BQ.png">

<sub>출처: https://towardsdatascience.com/understanding-rnn-and-lstm-f7cdf6dfc14e</sub>

## **1.1. 실습 : IMDB 데이터 with RNN**

### **IMDB 데이터 셋** ###
- 영화 리뷰 데이터
- 영화 리뷰, 긍정/부정 레이블로 구성(이진 분류)
- 텍스트 분석 작업에서 주로 활용
- 50000개의 리뷰 데이터 (Train 25000, Test 25000)


### **데이터 로드**

In [ ]:
# 설정 변수
num_words = 20000
max_len = 80
batch_size = 32

# imdb.npz 파일에서 데이터 로드
data = np.load('imdb.npz', allow_pickle=True)
x_train, y_train = data['x_train'], data['y_train']
x_test, y_test = data['x_test'], data['y_test']

print(len(x_train))  # 훈련 데이터 개수 출력
print(len(x_test))   # 테스트 데이터 개수 출력

# 시퀀스 길이를 max_len으로 패딩
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)

print(x_train.shape)  # 패딩된 훈련 데이터의 형태 출력
print(x_test.shape)   # 패딩된 테스트 데이터의 형태 출력

### **모델 구성**

In [ ]:
model = Sequential()
model.add(Embedding(num_words, 128))
model.add(SimpleRNN(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss = 'binary_crossentropy',
              metrics=['acc'])
model.summary()

### **모델 학습**

In [ ]:
history = model.fit(x_train, y_train,
                    epochs = 20,
                    batch_size = 128,
                    validation_split = 0.3)

### **시각화**

In [ ]:
import matplotlib.pyplot as plt
loss = history.history['loss']
val_loss = history.history['val_loss']
acc = history.history['acc']
val_acc = history.history['val_acc']

epochs = range(1, len(loss)+1)

plt.plot(epochs, loss, 'b--', label = 'training loss')
plt.plot(epochs, val_loss, 'r:', label = 'validation loss')
plt.grid()
plt.legend()

plt.figure()
plt.plot(epochs, acc, 'b--', label = 'training accuracy')
plt.plot(epochs, val_acc, 'r:', label = 'validation accuracy')
plt.grid()
plt.legend()

plt.show()

In [ ]:
model.evaluate(x_test, y_test)

**SimpleRNN의 한계점**

- SimpleRNN은 실전에 사용하기엔 너무 단순

- SimpleRNN은 이론적으로 시간 $t$ 에서 이전의 모든 타임스텝의 정보를 유지할 수 있지만, 실제로는 긴 시간에 걸친 의존성은 학습할 수 없음

- 그래디언트 소실 문제(vanishing gradient problem)
  - 이를 방지하기 위해 LSTM 등장



***

# **2. LSTM(Long Short-Term Memory)**
- 장단기 메모리 알고리즘

- 나중을 위해 정보를 저장함으로써 오래된 시그널이 점차 소실되는 것을 막아줌

  <img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png">

  <sub>출처: https://colah.github.io/posts/2015-08-Understanding-LSTMs/</sub>

## **2.1. 실습 : IMDB 데이터 with LSTM**

### **데이터 로드**

In [ ]:


# 설정 변수
num_words = 20000
max_len = 80
batch_size = 32

# imdb.npz 파일에서 데이터 로드
data = np.load('imdb.npz', allow_pickle=True)
x_train, y_train = data['x_train'], data['y_train']
x_test, y_test = data['x_test'], data['y_test']

print(len(x_train))  # 훈련 데이터 개수 출력
print(len(x_test))   # 테스트 데이터 개수 출력

# 시퀀스 길이를 max_len으로 패딩
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)

print(x_train.shape)  # 패딩된 훈련 데이터의 형태 출력
print(x_test.shape)   # 패딩된 테스트 데이터의 형태 출력

### **모델 구성**

In [ ]:
model = Sequential()

model.add(Embedding(num_words, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizer='rmsprop',
              loss = 'binary_crossentropy',
              metrics = ['acc'])
model.summary()

### **모델 학습**

In [ ]:
history = model.fit(x_train, y_train,
                    epochs = 20,
                    batch_size = 128,
                    validation_split=0.3)

### **시각화**

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
acc = history.history['acc']
val_acc = history.history['val_acc']

epochs = range(1, len(loss)+1)

plt.plot(epochs, loss, 'b--', label = 'training loss')
plt.plot(epochs, val_loss, 'r:', label = 'validation loss')
plt.grid()
plt.legend()

plt.figure()
plt.plot(epochs, acc, 'b--', label = 'training accuracy')
plt.plot(epochs, val_acc, 'r:', label = 'validation accuracy')
plt.grid()
plt.legend()

plt.show()

### **모델 평가**

In [ ]:
model.evaluate(x_test, y_test)

***

# **실습 1. Reuters(다중 분류)** 

## **Reuters**

- IMDB와 유사한 데이터셋(텍스트 데이터)
- 1986년 로이터에서 공개한 짧은 뉴스 기사 및 토픽의 집합
- 46개의 상호 배타적인 토픽으로 이루어진 데이터셋(다중 분류)


### **Reuters with RNN**

### 데이터셋 로드

In [ ]:
# 파라미터 설정
num_words = 10000
max_len = 500
batch_size = 32

# reuters.npz 파일에서 데이터 로드
data = np.load('reuters.npz', allow_pickle=True)

# 데이터로부터 x, y변수 추출
x = data['x']
y = data['y']

# 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# 데이터 차원 확인
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

### **데이터 전처리 및 확인**

In [ ]:
pad_x_train = pad_sequences(x_train, maxlen = max_len)
pad_x_test = pad_sequences(x_test, maxlen = max_len)
print(len(pad_x_train[0]))

### **모델 구성**

In [ ]:
model  =Sequential()
model.add(Embedding(input_dim = num_words, output_dim = 64))
model.add(SimpleRNN(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(46, activation = 'softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics = ['acc'])


model.summary()

In [ ]:
history = model.fit(pad_x_train, y_train,
                    batch_size = 32, epochs = 30,
                    validation_split=0.20)

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
acc = history.history['acc']
val_acc = history.history['val_acc']

epochs = range(1, len(loss)+1)

plt.plot(epochs, loss, 'b--', label = 'training loss')
plt.plot(epochs, val_loss, 'r:', label = 'validation loss')
plt.grid()
plt.legend()

plt.figure()
plt.plot(epochs, acc, 'b--', label = 'training accuracy')
plt.plot(epochs, val_acc, 'r:', label = 'validation accuracy')
plt.grid()
plt.legend()

plt.show()

In [ ]:
model.evaluate(pad_x_test, y_test)

***

# **실습2. 시계열 데이터 셋을 활용한 RNN**

### **Airline-passengers dataset**
- 1949년 1월 ~ 1960년 12월(144개월) 사이의 월간 비행기 이용객 수

In [ ]:
dataset = pd.read_csv('airline-passengers.csv', usecols=[1], engine='python')
plt.plot(dataset)
plt.show()

In [ ]:
np.random.seed(2020)

### **데이터 로드 및 전처리**

In [ ]:
# 데이터 불러오기
dataframe = pd.read_csv('airline-passengers.csv', usecols=[1], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')

# 데이터 정규화
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

# 데이터 분리
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

In [ ]:
# 배열의 값을 데이터셋 행렬로 변환
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

In [ ]:
# X=t 와 Y=t+1 로 변환
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

# 배열을 [samples, time steps, features] 형태로 변환
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

### **모델 구성**

In [ ]:
def build_model(n):
    model = Sequential()
    
    model.add(SimpleRNN(units = 32, activation='tanh', input_shape=(1,look_back)))
    model.add(Dense(1))
    
    model.compile(optimizer='adam',
                  loss='mse')
    
    return model

In [ ]:
model = build_model(10)
model.summary()

### **모델 학습**

In [ ]:
model.fit(trainX, trainY,
          epochs=100, batch_size=1, verbose=2)

In [ ]:
# 예측 수행
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# 예측 반전
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# MSE 산출
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
# 훈련 예측 값 plot을 위한 shift 수행
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# 테스트 예측 값 plot을 위한 shift 수행
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict

plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()